# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fd1d8b234f0>
├── 'a' --> tensor([[ 0.0497, -0.2613, -1.0780],
│                   [ 1.0702, -1.7239, -0.4826]])
└── 'x' --> <FastTreeValue 0x7fd1d8b23430>
    └── 'c' --> tensor([[ 0.1050,  1.3988, -0.0629,  0.6221],
                        [ 0.0488,  0.6910,  0.1519, -0.7510],
                        [ 0.0831, -0.0805, -0.6696,  1.2325]])

In [4]:
t.a

tensor([[ 0.0497, -0.2613, -1.0780],
        [ 1.0702, -1.7239, -0.4826]])

In [5]:
%timeit t.a

63.4 ns ± 0.0361 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fd1d8b234f0>
├── 'a' --> tensor([[-1.3751,  0.1744,  1.4436],
│                   [ 2.2543, -2.1718,  0.8170]])
└── 'x' --> <FastTreeValue 0x7fd1d8b23430>
    └── 'c' --> tensor([[ 0.1050,  1.3988, -0.0629,  0.6221],
                        [ 0.0488,  0.6910,  0.1519, -0.7510],
                        [ 0.0831, -0.0805, -0.6696,  1.2325]])

In [7]:
%timeit t.a = new_value

72 ns ± 0.0535 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.0497, -0.2613, -1.0780],
               [ 1.0702, -1.7239, -0.4826]]),
    x: Batch(
           c: tensor([[ 0.1050,  1.3988, -0.0629,  0.6221],
                      [ 0.0488,  0.6910,  0.1519, -0.7510],
                      [ 0.0831, -0.0805, -0.6696,  1.2325]]),
       ),
)

In [10]:
b.a

tensor([[ 0.0497, -0.2613, -1.0780],
        [ 1.0702, -1.7239, -0.4826]])

In [11]:
%timeit b.a

59.2 ns ± 0.0751 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.3270, -1.1534, -0.0359],
               [-0.6373,  1.0504, -0.7586]]),
    x: Batch(
           c: tensor([[ 0.1050,  1.3988, -0.0629,  0.6221],
                      [ 0.0488,  0.6910,  0.1519, -0.7510],
                      [ 0.0831, -0.0805, -0.6696,  1.2325]]),
       ),
)

In [13]:
%timeit b.a = new_value

498 ns ± 0.15 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

851 ns ± 0.787 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.9 µs ± 11.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

144 µs ± 624 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

146 µs ± 1.59 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fd1d8b23040>
├── 'a' --> tensor([[[ 0.0497, -0.2613, -1.0780],
│                    [ 1.0702, -1.7239, -0.4826]],
│           
│                   [[ 0.0497, -0.2613, -1.0780],
│                    [ 1.0702, -1.7239, -0.4826]],
│           
│                   [[ 0.0497, -0.2613, -1.0780],
│                    [ 1.0702, -1.7239, -0.4826]],
│           
│                   [[ 0.0497, -0.2613, -1.0780],
│                    [ 1.0702, -1.7239, -0.4826]],
│           
│                   [[ 0.0497, -0.2613, -1.0780],
│                    [ 1.0702, -1.7239, -0.4826]],
│           
│                   [[ 0.0497, -0.2613, -1.0780],
│                    [ 1.0702, -1.7239, -0.4826]],
│           
│                   [[ 0.0497, -0.2613, -1.0780],
│                    [ 1.0702, -1.7239, -0.4826]],
│           
│                   [[ 0.0497, -0.2613, -1.0780],
│                    [ 1.0702, -1.7239, -0.4826]]])
└── 'x' --> <FastTreeValue 0x7fd1d8b237f0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.7 µs ± 59.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fd100c39fd0>
├── 'a' --> tensor([[ 0.0497, -0.2613, -1.0780],
│                   [ 1.0702, -1.7239, -0.4826],
│                   [ 0.0497, -0.2613, -1.0780],
│                   [ 1.0702, -1.7239, -0.4826],
│                   [ 0.0497, -0.2613, -1.0780],
│                   [ 1.0702, -1.7239, -0.4826],
│                   [ 0.0497, -0.2613, -1.0780],
│                   [ 1.0702, -1.7239, -0.4826],
│                   [ 0.0497, -0.2613, -1.0780],
│                   [ 1.0702, -1.7239, -0.4826],
│                   [ 0.0497, -0.2613, -1.0780],
│                   [ 1.0702, -1.7239, -0.4826],
│                   [ 0.0497, -0.2613, -1.0780],
│                   [ 1.0702, -1.7239, -0.4826],
│                   [ 0.0497, -0.2613, -1.0780],
│                   [ 1.0702, -1.7239, -0.4826]])
└── 'x' --> <FastTreeValue 0x7fd110cffe20>
    └── 'c' --> tensor([[ 0.1050,  1.3988, -0.0629,  0.6221],
                        [ 0.0488,  0.6910,  0.1519, -0.7510],
                 

In [23]:
%timeit t_cat(trees)

31.9 µs ± 63.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

66.2 µs ± 69.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.0497, -0.2613, -1.0780],
                [ 1.0702, -1.7239, -0.4826]],
       
               [[ 0.0497, -0.2613, -1.0780],
                [ 1.0702, -1.7239, -0.4826]],
       
               [[ 0.0497, -0.2613, -1.0780],
                [ 1.0702, -1.7239, -0.4826]],
       
               [[ 0.0497, -0.2613, -1.0780],
                [ 1.0702, -1.7239, -0.4826]],
       
               [[ 0.0497, -0.2613, -1.0780],
                [ 1.0702, -1.7239, -0.4826]],
       
               [[ 0.0497, -0.2613, -1.0780],
                [ 1.0702, -1.7239, -0.4826]],
       
               [[ 0.0497, -0.2613, -1.0780],
                [ 1.0702, -1.7239, -0.4826]],
       
               [[ 0.0497, -0.2613, -1.0780],
                [ 1.0702, -1.7239, -0.4826]]]),
    x: Batch(
           c: tensor([[[ 0.1050,  1.3988, -0.0629,  0.6221],
                       [ 0.0488,  0.6910,  0.1519, -0.7510],
                       [ 0.0831, -0.0805, -0.6696,  1.2325]],
         

In [26]:
%timeit Batch.stack(batches)

79.3 µs ± 145 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.0497, -0.2613, -1.0780],
               [ 1.0702, -1.7239, -0.4826],
               [ 0.0497, -0.2613, -1.0780],
               [ 1.0702, -1.7239, -0.4826],
               [ 0.0497, -0.2613, -1.0780],
               [ 1.0702, -1.7239, -0.4826],
               [ 0.0497, -0.2613, -1.0780],
               [ 1.0702, -1.7239, -0.4826],
               [ 0.0497, -0.2613, -1.0780],
               [ 1.0702, -1.7239, -0.4826],
               [ 0.0497, -0.2613, -1.0780],
               [ 1.0702, -1.7239, -0.4826],
               [ 0.0497, -0.2613, -1.0780],
               [ 1.0702, -1.7239, -0.4826],
               [ 0.0497, -0.2613, -1.0780],
               [ 1.0702, -1.7239, -0.4826]]),
    x: Batch(
           c: tensor([[ 0.1050,  1.3988, -0.0629,  0.6221],
                      [ 0.0488,  0.6910,  0.1519, -0.7510],
                      [ 0.0831, -0.0805, -0.6696,  1.2325],
                      [ 0.1050,  1.3988, -0.0629,  0.6221],
                      [ 0.0488,  

In [28]:
%timeit Batch.cat(batches)

147 µs ± 231 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

346 µs ± 2.11 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
